# FAISS 인덱스 제작
이 노트북은 단계별로 이미지를 생성/불러와 OCR 모델로 벡터를 추출하고 FAISS 인덱스를 만드는 예시입니다.
각 코드 셀을 순서대로 실행하세요.

In [1]:
%pip install numpy pillow tensorflow faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [7]:
# 라이브러리 임포트 및 스크립트 불러오기
import os
import numpy as np
import tensorflow as tf
import faiss
import gc

In [8]:
# inline helper 함수들 정의
import math
from PIL import Image, ImageDraw, ImageFont

# faiss는 나중에 설치되어야 합니다. 노트북 상단에서 설치 커맨드를 실행하세요.
try:
    import faiss
except Exception:
    faiss = None

import numpy as np
import tensorflow as tf

# 한글 음절 분해/평가용 슬라이스 (모델 출력 인덱스에 맞춤)
L_SLICE = slice(0, 19)
V_SLICE = slice(19, 40)
T_SLICE = slice(40, 68)

# 손실/지표 함수 (모델 로드에 필요할 수 있음)
def hangul_loss(y_true, logits):
    cho  = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, L_SLICE], logits=logits[:, L_SLICE])
    jung = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, V_SLICE], logits=logits[:, V_SLICE])
    jong = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, T_SLICE], logits=logits[:, T_SLICE])
    return cho + jung + jong


def acc_first(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1)), tf.float32))

def acc_middle(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1)), tf.float32))

def acc_last(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1)), tf.float32))

def acc_joint(y_true, y_pred):
    l_ok = tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1))
    v_ok = tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1))
    t_ok = tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1))
    return tf.reduce_mean(tf.cast(l_ok & v_ok & t_ok, tf.float32))


# 글자 렌더링 함수 (노트북 환경에서 바로 사용)
def render_char_to_tensor(char: str, image_size=(128,128), font_path: str = None, font_size: int = 96):
    W, H = image_size
    try:
        if font_path and os.path.exists(font_path):
            font = ImageFont.truetype(font_path, font_size)
        else:
            font = ImageFont.load_default()
    except Exception:
        font = ImageFont.load_default()

    image = Image.new("L", (W, H), "white")
    draw = ImageDraw.Draw(image)

    try:
        bbox = draw.textbbox((0, 0), char, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        position = ((W - text_width) / 2 - bbox[0], (H - text_height) / 2 - bbox[1])
    except Exception:
        position = (5,5)

    draw.text(position, char, font=font, fill="black")
    arr = np.array(image, dtype=np.float32) / 255.0
    return arr.reshape((H, W, 1))


# 텐서 로드 또는 코드포인트 범위로부터 생성
def load_or_generate_tensors(tensors_path, labels_path):
    if tensors_path and os.path.exists(tensors_path):
        print(f"Loading tensors from {tensors_path}")
        X = np.load(tensors_path)
        if labels_path and os.path.exists(labels_path):
            labels = np.load(labels_path, allow_pickle=True)
        else:
            raise ValueError("Labels file required when providing tensors. Use --labels <file>.")
        return X, labels


# 모델로부터 벡터를 추출하는 함수 (배치 처리)
def extract_vectors(model, X: np.ndarray, batch_size: int = 256, encoder=None):
    N = X.shape[0]
    parts = []
    for i in range(0, N, batch_size):
        batch = X[i:i+batch_size].astype(np.float32)
        preds = model.predict(batch, verbose=0)
        if encoder is not None:
            preds = encoder.predict(preds, verbose=0)
        parts.append(preds.astype(np.float32))
    vectors = np.concatenate(parts, axis=0)
    return vectors


# FAISS 인덱스 빌드 함수
def ensure_faiss():
    if faiss is None:
        raise RuntimeError("faiss not available. Install with 'pip install faiss-cpu'.")


def build_index(vectors: np.ndarray, mapping: np.ndarray, out_index_path: str, nprobe: int = 20):
    ensure_faiss()
    n, d = vectors.shape
    print(f"Building FAISS index for {n} vectors, dim={d}")
    if n > 5000:
        nlist = int(max(64, math.sqrt(n)))
        quantizer = faiss.IndexFlatL2(d)
        index = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)
        print(f"Training IVF index with nlist={nlist}...")
        index.train(vectors)
        index.add(vectors)
    else:
        index = faiss.IndexFlatL2(d)
        index.add(vectors)
    try:
        index.nprobe = nprobe
    except Exception:
        pass
    faiss.write_index(index, out_index_path)
    print(f"Saved FAISS index to {out_index_path}")

## 1) 이미지 텐서 생성 또는 불러오기
- 기존 `.npy` 텐서가 있으면 해당 파일을 사용하세요.
- 없으면 코드포인트 범위를 지정해 폰트로 렌더링하여 생성합니다.

In [9]:
# 텐서 생성/불러기 설정 예시
TENSORS_PATH = "./hangul_tensors.npy" 
LABELS_PATH = "./hangul_labels.npy" 

## 2) OCR 모델 로드
- 학습한 `final.keras` 모델 파일 경로를 지정하세요.

In [10]:
#OCR 모델 로드
MODEL_PATH = "./final.keras"

L_SLICE = slice(0, 19)
V_SLICE = slice(19, 40)
T_SLICE = slice(40, 68)

def hangul_loss(y_true, logits):
    cho  = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, L_SLICE], logits=logits[:, L_SLICE])
    jung = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, V_SLICE], logits=logits[:, V_SLICE])
    jong = tf.nn.softmax_cross_entropy_with_logits(labels=y_true[:, T_SLICE], logits=logits[:, T_SLICE])
    return cho + jung + jong

def acc_first(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, L_SLICE], -1),
                                           tf.argmax(y_pred[:, L_SLICE], -1)), tf.float32))
def acc_middle(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, V_SLICE], -1),
                                           tf.argmax(y_pred[:, V_SLICE], -1)), tf.float32))
def acc_last(y_true, y_pred):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_true[:, T_SLICE], -1),
                                           tf.argmax(y_pred[:, T_SLICE], -1)), tf.float32))
def acc_joint(y_true, y_pred):
    l_ok = tf.equal(tf.argmax(y_true[:, L_SLICE], -1), tf.argmax(y_pred[:, L_SLICE], -1))
    v_ok = tf.equal(tf.argmax(y_true[:, V_SLICE], -1), tf.argmax(y_pred[:, V_SLICE], -1))
    t_ok = tf.equal(tf.argmax(y_true[:, T_SLICE], -1), tf.argmax(y_pred[:, T_SLICE], -1))
    return tf.reduce_mean(tf.cast(l_ok & v_ok & t_ok, tf.float32))

# --- 모델 로드 ---
model = tf.keras.models.load_model(
    MODEL_PATH,
    custom_objects={
        "hangul_loss": hangul_loss,
        "acc_first": acc_first,
        "acc_middle": acc_middle,
        "acc_last": acc_last,
        "acc_joint": acc_joint,
    }
)



def ocr(character):
    X = character                               # shape: (N, 128,128,1)

    if X.ndim == 3:
        X = np.expand_dims(X, -1)
    X = X.astype(np.float32)
    if np.max(X) > 1.5:
        X /= 255.0

    # 모델 예측
    preds = model.predict(X, batch_size=256, verbose=1)  # (N,68)

    return preds



c:\Users\stayup24h\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 320 variables whereas the saved optimizer has 324 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## 3) 벡터 추출
- OCR 모델 출력(예: 68차원)을 그대로 사용하거나, 필요하면 인코더를 통해 차원 축소합니다.

In [12]:
# 벡터 추출 예시
BATCH_SIZE = 256
encoder_model_path = None  # 예: './best_simple_encoder_for_faiss.h5'
encoder = None
if encoder_model_path:
    encoder = tf.keras.models.load_model(encoder_model_path)

X = np.load(TENSORS_PATH)

# Initialize an empty Python list to store predictions
predictions_list = []

for i in range(X.shape[0]):
    image = np.expand_dims(X[i], 0)
    pred = ocr(image)
    predictions_list.append(pred)
    
    if i % 1000 == 0:
        del image
        gc.collect()
        print(f"Processed {i} / {X.shape[0]} images")   
    
    
# Convert the list of predictions into a single NumPy array
vectors = np.concatenate(predictions_list, axis=0)

print('벡터 형상:', vectors.shape)
np.save('unicode_vectors.npy', vectors)
print('벡터 저장: unicode_vectors.npy')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Processed 0 / 11172 images
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━

## 4) FAISS 인덱스 생성 및 저장
- 데이터 수에 따라 Flat 또는 IVF 인덱스를 자동으로 선택합니다.
- `nprobe`는 검색시 탐색할 클러스터 개수로, 정확도/속도 조정을 위해 변경 가능합니다.

In [14]:
# FAISS 인덱스 빌드
OUT_INDEX = 'hangle_faiss_index.faiss'
OUT_MAP = 'faiss_hangle_map.npy'
NPROBE = 20

FAISS_INDEX_FILENAME = 'hangle_faiss_index.faiss'

# --- FAISS 인덱스 설정 ---
NLIST = 100  # 클러스터(Partition) 개수. (데이터 개수의 sqrt 근처 값 권장)

final_unicode_X = np.load('unicode_vectors.npy')
final_unicode_X = np.array(final_unicode_X)

D = final_unicode_X

# 1. Quantizer 정의: 벡터를 클러스터링할 때 사용할 기준 (일반적으로 L2 거리를 사용하는 Flat 인덱스 사용)
# 'D' 대신 벡터의 차원 (D.shape[1])을 전달해야 합니다.
quantizer = faiss.IndexFlatL2(D.shape[1])

# 2. IVF 인덱스 생성: D차원, NLIST개의 클러스터, L2 거리 사용
# 'D' 대신 벡터의 차원 (D.shape[1])을 전달해야 합니다.
index = faiss.IndexIVFFlat(quantizer, D.shape[1], NLIST, faiss.METRIC_L2)

# 3. 인덱스 훈련 (Training)
# IVF 인덱스는 add 전에 클러스터 중심점을 계산하는 훈련이 필요합니다.
print("FAISS Index Training Start...")
index.train(final_unicode_X)
print("Training Complete. Index is trained:", index.is_trained)

# 4. 벡터 추가 (Adding)
# 16차원 잠재 벡터를 인덱스에 추가합니다.
index.add(final_unicode_X)
print(f"Total vectors added: {index.ntotal}")

# 5. 검색 속도 vs. 정확도 설정 (nprobe)
# 검색 시 확인할 클러스터의 개수. 높을수록 정확하나 느립니다.
# 20만 개에서는 10~30 사이가 적절하며, 20을 추천합니다.
index.nprobe = 20

# 6. 인덱스 저장
faiss.write_index(index, FAISS_INDEX_FILENAME)
print(f"FAISS Index saved as {FAISS_INDEX_FILENAME}")

FAISS Index Training Start...
Training Complete. Index is trained: True
Total vectors added: 11172
FAISS Index saved as hangle_faiss_index.faiss


## 5) 간단 검색 예시
- 생성한 FAISS 인덱스와 매핑을 불러와 샘플 문자로 검색해봅니다.

In [17]:
import faiss

index = faiss.read_index(OUT_INDEX)
u_map = np.load("E:\github\Hangul-to-Unicode-Obfuscation-Project\Hangul_to_unicode_obfuscater\hangul_labels.npy", allow_pickle=True)

# 예시 입력 문자
query_char = '왓'
q_tensor = render_char_to_tensor(query_char, font_path=FONT_PATH)
q_tensor = np.expand_dims(q_tensor, 0).astype(np.float32)
q_vec = model.predict(q_tensor)
if encoder is not None:
    q_vec = encoder.predict(q_vec)

print(q_vec)
D, I = index.search(q_vec, 10)
print('Top results:')
for rank, (idx, dist) in enumerate(zip(I[0], D[0])):
    if idx >= 0:
        print(f"{rank+1}. codepoint={u_map[idx]} char={chr(int(u_map[idx]))} dist={dist}")

<>:4: SyntaxWarning: invalid escape sequence '\g'
<>:4: SyntaxWarning: invalid escape sequence '\g'
C:\Users\stayup24h\AppData\Local\Temp\ipykernel_27852\3963746706.py:4: SyntaxWarning: invalid escape sequence '\g'
  u_map = np.load("E:\github\Hangul-to-Unicode-Obfuscation-Project\Hangul_to_unicode_obfuscater\hangul_labels.npy", allow_pickle=True)
C:\Users\stayup24h\AppData\Local\Temp\ipykernel_27852\3963746706.py:4: SyntaxWarning: invalid escape sequence '\g'
  u_map = np.load("E:\github\Hangul-to-Unicode-Obfuscation-Project\Hangul_to_unicode_obfuscater\hangul_labels.npy", allow_pickle=True)


NameError: name 'FONT_PATH' is not defined